In [ ]:
from panda import Panda

panda = Panda()
panda.set_safety_mode(Panda.SAFETY_ELM327)

In [ ]:
from panda.python.uds import UdsClient, MessageTimeoutError, NegativeResponseError, InvalidSubAddressError, \
                             SESSION_TYPE, DATA_IDENTIFIER_TYPE

print('Waiting for ignition...')
while True:
  health = panda.health()
  print(health)
  if health['ignition_line']:
    break

print('Ignition detected')

addr = 0x730  # PSCM
sub_addr = None
rx_addr = addr + 0x8
bus = 0

uds_client = UdsClient(panda, addr, rx_addr, bus, sub_addr=sub_addr, timeout=0.2, debug=True)

while True:
  try:
    uds_client.tester_present()
    uds_client.diagnostic_session_control(SESSION_TYPE.DEFAULT)
    uds_client.diagnostic_session_control(SESSION_TYPE.EXTENDED_DIAGNOSTIC)
  except NegativeResponseError:
    pass
  except MessageTimeoutError:
    print('Timeout waiting for response')
  except InvalidSubAddressError as e:
    print(f'Invalid sub-address: {e}')